In [1]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

0

In [2]:
import torch
roberta = torch.hub.load('pytorch/fairseq', 'roberta.base')
roberta.eval()  # disable dropout (or leave in train mode to finetune)

Using cache found in /home/a.tliamov/.cache/torch/hub/pytorch_fairseq_main
2023-03-26 18:50:23 | INFO | fairseq.file_utils | loading archive file http://dl.fbaipublicfiles.com/fairseq/models/roberta.base.tar.gz from cache at /home/a.tliamov/.cache/torch/pytorch_fairseq/37d2bc14cf6332d61ed5abeb579948e6054e46cc724c7d23426382d11a31b2d6.ae5852b4abc6bf762e0b6b30f19e741aa05562471e9eb8f4a6ae261f04f9b350
/home/a.tliamov/anaconda3/envs/my_env/lib/python3.6/site-packages/hydra/experimental/initialize.py:43: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  deprecation_warning(message=message)
/home/a.tliamov/anaconda3/envs/my_env/lib/python3.6/site-packages/hydra/experimental/initialize.py:48: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  caller_stack_depth=caller_stack_depth + 1,
/home/a.tliamov/anaconda3/envs/my_env/lib/python3.6/site-package

RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(50265, 768, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 768, padding_idx=1)
        (layernorm_embedding): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0): TransformerEncoderLayerBase(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout_module

In [3]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
from tqdm import tqdm
import gensim.downloader as api


data_meta = []
with gzip.open('meta_Movies_and_TV.json.gz') as f:
    for l in tqdm(f):
        data_meta.append(json.loads(l.strip()))

df_meta = pd.DataFrame.from_dict(data_meta)
data = df_meta[["asin", "description" , "also_buy"]]
data.head()

/home/a.tliamov/anaconda3/envs/my_env/lib/python3.6/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
203766it [00:09, 20981.46it/s]


,asin,description,also_buy
0,0000695009,[],[]
1,0000791156,[],[]
2,0000143529,[Disc 1: Flour Power (Scones; Shortcakes; Sout...,[]
3,0000143588,[Barefoot Contessa Volume 2: On these three di...,"[B002I5GNW4, B005WXPVMM, B009UY3W8O, B00N27ID1..."
4,0000143502,[Rise and Swine (Good Eats Vol. 7) includes bo...,"[B000P1CKES, B000NR4CRM]"


In [4]:
data["description"] = data["description"].apply(lambda x: ". ".join(x))
data.head()

/home/a.tliamov/anaconda3/envs/my_env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,asin,description,also_buy
0,0000695009,,[]
1,0000791156,,[]
2,0000143529,Disc 1: Flour Power (Scones; Shortcakes; South...,[]
3,0000143588,Barefoot Contessa Volume 2: On these three dis...,"[B002I5GNW4, B005WXPVMM, B009UY3W8O, B00N27ID1..."
4,0000143502,Rise and Swine (Good Eats Vol. 7) includes bon...,"[B000P1CKES, B000NR4CRM]"


In [5]:
data = data[data["description"].apply(len) > 0][["asin", "description"]]
roberta.to("cuda")
roberta.eval()

from tqdm import tqdm
torch.cuda.memory_summary(device=None, abbreviated=False)

def func(text):
    tokens = roberta.encode(text)
    last_layer_features = roberta.extract_features(tokens[:512])
    del tokens
    r = torch.mean(last_layer_features, axis=1)
    del last_layer_features
    return r.detach().cpu()
l = []
for text in tqdm(data["description"].to_list()):
    l.append(func(text))

# data["emb"] = data["description"].apply(lambda x: func(x))


100%|██████████| 174773/174773 [38:37<00:00, 75.42it/s]


In [11]:
l1 =[j.numpy() for j in l]
data["emb"] = l1

In [23]:
l1[0].shape

(1, 768)

In [12]:
import numpy as np
train_data = data.groupby("asin")['emb'].apply(np.mean)

In [25]:
train_data

asin
0000143502    [[0.033036567, 0.08688408, 0.03734081, -0.1264...
0000143529    [[0.03975344, 0.047156136, 0.06837927, -0.1605...
0000143561    [[0.07635365, 0.085783996, -0.00197069, 0.0210...
0000143588    [[0.047178604, 0.13676795, 0.027243488, -0.014...
000073991X    [[-0.00826823, 0.07092879, 0.06727121, -0.1449...
                                    ...                        
B01HJ1INB0    [[-0.025023045, 0.03975451, -0.0072285887, -0....
B01HJ3E0PQ    [[-0.042048983, 0.15844019, -0.008411095, 0.00...
B01HJ6R77G    [[-0.023473285, 0.035423875, 0.04001624, -0.04...
B01HJCCLOY    [[0.02384126, 0.064528406, 0.038299423, -0.157...
B01HJF79XO    [[0.008206782, 0.13220622, 0.0781732, 0.027552...
Name: emb, Length: 156728, dtype: object

In [26]:
train_data = train_data.apply(lambda x: x.squeeze())

In [29]:

rrr = pd.DataFrame(train_data)
rrr["emb"]

asin
0000143502    [0.033036567, 0.08688408, 0.03734081, -0.12641...
0000143529    [0.03975344, 0.047156136, 0.06837927, -0.16053...
0000143561    [0.07635365, 0.085783996, -0.00197069, 0.02101...
0000143588    [0.047178604, 0.13676795, 0.027243488, -0.0148...
000073991X    [-0.00826823, 0.07092879, 0.06727121, -0.14498...
                                    ...                        
B01HJ1INB0    [-0.025023045, 0.03975451, -0.0072285887, -0.0...
B01HJ3E0PQ    [-0.042048983, 0.15844019, -0.008411095, 0.005...
B01HJ6R77G    [-0.023473285, 0.035423875, 0.04001624, -0.048...
B01HJCCLOY    [0.02384126, 0.064528406, 0.038299423, -0.1571...
B01HJF79XO    [0.008206782, 0.13220622, 0.0781732, 0.0275526...
Name: emb, Length: 156728, dtype: object

In [30]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

neigh = NearestNeighbors(n_neighbors=100, radius=0.4, metric="cosine")
neigh.fit(rrr["emb"].tolist())

NearestNeighbors(metric='cosine', n_neighbors=100, radius=0.4)

In [32]:
data2 = df_meta[["asin", "description" , "also_buy"]]
metric_data = data2[data2["also_buy"].apply(len) > 0][["asin", "also_buy"]]
metric_asins = set(metric_data["asin"].tolist())
metric_asins

{'B001152THA',
 'B000VI70Y0',
 'B00K423A7O',
 'B0018UZ3UI',
 'B004ULEEOI',
 'B009KKLQ5O',
 'B00Q6YLPX0',
 'B00VJIIKG8',
 'B005LVEFVS',
 'B006H4RAS8',
 'B00THD5QGE',
 'B00009VTXM',
 'B002PHTY5I',
 'B00006JDVT',
 'B000NDDU0G',
 'B00507IBTA',
 'B005E7AODW',
 'B00GMM19X2',
 'B00HUAGZ34',
 'B005QBST4C',
 'B0009VQ7N4',
 'B000TSTEM8',
 'B004UIFZQC',
 'B00OG0EL1E',
 'B000VA2YDK',
 'B009NPBBTC',
 'B00684A318',
 '6305870861',
 'B00005BSM8',
 'B000YYWO38',
 'B002XTXG8Y',
 'B0052OYEFG',
 'B0099116J0',
 'B00A6UHC0U',
 'B001O4C6SA',
 'B000L42NNW',
 'B002WNU0OY',
 'B004T1RAYA',
 'B00EF0NYA4',
 'B00SMTVHU8',
 'B00DYC6N20',
 'B00F6IMD4C',
 'B00CPL8C3O',
 'B001CQS7SI',
 'B0031XYLRQ',
 'B001O0J682',
 'B00699G65E',
 'B004ZG17G6',
 'B00SVEXKAO',
 '9089702601',
 'B005P3O03K',
 'B00EUILFQ6',
 'B002CWKTQ4',
 'B000E0LLKY',
 'B00171SVHK',
 'B008YYSEBG',
 'B00080HHO4',
 'B000I0QLG4',
 'B0076FEH4W',
 '1943535396',
 'B011VQNXGE',
 'B000BHH6BI',
 'B006FKNBSC',
 '0792846133',
 'B0001Z48SE',
 'B001IZNJ3G',
 'B001N4K6

In [33]:
correct_asins = list(set(rrr.index.tolist() ) & set(metric_asins))

In [34]:
res1 = neigh.kneighbors(rrr.loc[correct_asins]["emb"].tolist(), 5, return_distance=True)
res1

(array([[0.0000000e+00, 4.9476624e-03, 5.3820014e-03, 5.6831837e-03,
         5.7921410e-03],
        [5.9604645e-08, 7.9962015e-03, 8.3372593e-03, 8.3606243e-03,
         8.4453225e-03],
        [1.1920929e-07, 7.3218942e-03, 8.3941221e-03, 8.4632635e-03,
         8.6803436e-03],
        ...,
        [0.0000000e+00, 3.9684772e-03, 4.0010214e-03, 4.3442249e-03,
         4.4133663e-03],
        [5.9604645e-08, 5.6288838e-03, 6.3513517e-03, 6.6743493e-03,
         7.0161819e-03],
        [0.0000000e+00, 1.4580727e-02, 1.6486228e-02, 1.6667366e-02,
         1.7351627e-02]], dtype=float32),
 array([[ 70907,  78610,  80498,  21353,  71627],
        [ 66859,  12862,  80927,  63772, 138565],
        [139734, 134012, 132089,  67547, 131725],
        ...,
        [152183,  56767,  75923, 148146, 151502],
        [ 69654,  84547,  60000,  68080, 103727],
        [ 60077,  40578,  22103,  18831,  24123]]))

In [37]:
all_asins = rrr.index.tolist()
predict_items = list(np.array(all_asins)[res1[1].tolist()])

result = pd.DataFrame({"asin": rrr.loc[correct_asins].index.tolist(), "predict_items": predict_items})
result.head()

,asin,predict_items
0,B001152THA,"[B001152THA, B001BP14SK, B001DN0US0, B00005M1Z..."
1,B000VI70Y0,"[B000VI70Y0, 6305004218, B001E52JIG, B000RKDEW..."
2,B00K423A7O,"[B00K423A7O, B00FXCIP7U, B00EJJGDMM, B000WCN8R..."
3,B0018UZ3UI,"[B0018UZ3UI, B001EC857I, B0019D6TAW, B0065IYRN..."
4,B004ULEEOI,"[B004ULEEOI, B00DSAUH0W, B00HLT0YKO, B001CUBAT..."


In [38]:
df_metric = metric_data.merge(result, on="asin")
df_metric.head()

,asin,also_buy,predict_items
0,0000143588,"[B002I5GNW4, B005WXPVMM, B009UY3W8O, B00N27ID1...","[B001CSGJN6, 0000143588, B000NR7ROM, B000KZZ6W..."
1,0000143502,"[B000P1CKES, B000NR4CRM]","[0000143502, B0015SVNXY, B0019BK3KQ, B000UMMA8..."
2,0000143561,"[B002I5GNVU, B000KL8ODE]","[0000143561, B002TCRQ68, B000HC2LYE, B000UMMB5..."
3,0001517791,[B000EK77AM],"[0001517791, B000EGO2R2, B000BHK9BM, B005KBMSU..."
4,0005089549,[B00M45SW04],"[0005089549, 6303103340, 6304252137, 630302929..."


In [39]:
df_metric.apply(lambda x: len(list(set(x["also_buy"]) & set(x["predict_items"])))/4, axis=1).mean()

0.06315050805591423

In [40]:
df_metric.apply(lambda x: len(list(set(x["also_buy"]) & set(x["predict_items"])))/len(set(x["also_buy"])), axis=1).mean()


0.02154282835420157

In [41]:
df_metric["predict_items"][0].tolist()

['B001CSGJN6', '0000143588', 'B000NR7ROM', 'B000KZZ6WM', '6302526027']

In [61]:

data3.loc[df_metric["predict_items"][0].tolist()]["description"].to_list()

['Barefoot Contessa Volume 2: On these three discs, "Brunch N Lunch", "Picnic Parties", and "Summer Entertaining", Ina shows you the ins and outs of casual yet elegant entertaining. Learn to prepare scrumptious dishes for all occasions, from birthday parties and picnics to beach parties and brunch gatherings at home. Recipes include Endive and Avocado Salad, Meringues with Cassis and Raspberries, Affogato, Caesar Salad with Pancetta, Pissaladiere, Lemon Cake, Marmalade Butter, Potato Basil Frittata, Cheddar Corn Chowder, and many more delicious offerings....210 minutes run time',
 'Barefoot Contessa Volume 2: On these three discs, "Brunch N Lunch", "Picnic Parties", and "Summer Entertaining", Ina shows you the ins and outs of casual yet elegant entertaining. Learn to prepare scrumptious dishes for all occasions, from birthday parties and picnics to beach parties and brunch gatherings at home. Recipes include Endive and Avocado Salad, Meringues with Cassis and Raspberries, Affogato, Cae

In [58]:
data3 = data.set_index("asin")